# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298621972621                   -0.85    5.0         
  2   -8.300231952805       -2.79       -1.25    1.0    177ms
  3   -8.300440039160       -3.68       -1.89    2.0    204ms
  4   -8.300461341401       -4.67       -2.75    3.0    417ms
  5   -8.300463953155       -5.58       -3.07    2.8    233ms
  6   -8.300464294706       -6.47       -3.26    9.2    351ms
  7   -8.300464471968       -6.75       -3.41    1.8    203ms
  8   -8.300464568651       -7.01       -3.56    2.1    230ms
  9   -8.300464626612       -7.24       -3.77    1.5    253ms
 10   -8.300464637281       -7.97       -3.93    1.4    178ms
 11   -8.300464643726       -8.19       -4.28    1.9    194ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67394561144                   -0.70    6.2         
  2   -16.67796519227       -2.40       -1.14    1.8    457ms
  3   -16.67914304701       -2.93       -1.86    3.4    551ms
  4   -16.67924735738       -3.98       -2.65    3.2    557ms
  5   -16.67928460329       -4.43       -3.08    3.6    612ms
  6   -16.67928619559       -5.80       -3.47    3.8    549ms
  7   -16.67928621108       -7.81       -3.98    2.5    506ms
  8   -16.67928622133       -7.99       -4.59    2.5    490ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32528650909                   -0.56    6.4         
  2   -33.33256945523       -2.14       -1.00    1.0    1.52s
  3   -33.33405669182       -2.83       -1.75    4.1    1.95s
  4   -33.33426400863       -3.68       -2.64    4.2    1.88s
  5   -33.33610791309       -2.73       -2.44    4.5    2.20s
  6   -33.33694242377       -3.08       -2.52    7.9    2.17s
  7   -33.33694232426   +   -7.00       -2.52    1.9    1.50s
  8   -33.33676145360   +   -3.74       -2.21    2.8    2.06s
  9   -33.33669696443   +   -4.19       -2.14    1.0    1.47s
 10   -33.33670326362       -5.20       -2.15    1.0    1.48s
 11   -33.33671699001       -4.86       -2.16    1.0    1.49s
 12   -33.33674014325       -4.64      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298268771986                   -0.85    5.1         
  2   -8.300238496191       -2.71       -1.59    1.0    164ms
  3   -8.300422672664       -3.73       -2.57    3.2    202ms
  4   -8.300383945124   +   -4.41       -2.30    3.8    242ms
  5   -8.300464162874       -4.10       -3.39    1.0    140ms
  6   -8.300464557823       -6.40       -3.74    4.0    254ms
  7   -8.300464638778       -7.09       -4.24    3.9    253ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32653379053                   -0.56    7.0         
  2   -33.31649073521   +   -2.00       -1.27    1.5    1.37s
  3   -15.42070715287   +    1.25       -0.44    6.4    2.87s
  4   -33.33032577719        1.25       -1.97    5.2    2.45s
  5   -33.21569582581   +   -0.94       -1.35    4.1    2.43s
  6   -32.94934970602   +   -0.57       -1.27    4.8    2.44s
  7   -33.32251328838       -0.43       -1.92    4.8    2.21s
  8   -33.33573190204       -1.88       -2.46    2.8    1.58s
  9   -33.33661905843       -3.05       -2.50    3.8    2.03s
 10   -33.33656045232   +   -4.23       -2.73    1.9    1.56s
 11   -33.33688853926       -3.48       -3.10    2.2    1.51s
 12   -33.33692295458       -4.46      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.